In [ ]:


str = ['100111011000100111000011011111101111111111111001111111111100', '100010101010101001101001100101001001010101010100100101110001', '111100111100011100011100001100101110011110001111000111000111', '010101010100111011101101101100011010110001000011100001011100']
n = 60
N = 4
X = []
for i in range(N):
    X.append([int(str[i][j]) for j in range(n)])

k = []
for i in range(N):
    s = 0
    for j in range(n):
        if X[i][j] == 0:
            s+=1
    k.append(s) #number of zeros

m = [n - k[i] for i in range(N)] #number of ones

print('ones & zeros')
print(m,k)







ones & zeros
[42, 27, 34, 30] [18, 33, 26, 30]


In [ ]:
from scipy.stats import norm
from math import sqrt
#1 по критерию серий отделим Бернулли от не Бернулли
T_series_cr = []
for i in range(N):
    s = 0
    for j in range(n-1):
        if X[i][j] != X[i][j+1]:
            s += 1
    T_series_cr.append((s + 1 - 2*k[i]*m[i]/n - 1)/sqrt(2*k[i]*m[i]*(2*k[i]*m[i] - n)/(n*n*(n-1))))

print('Значения статистик: ', T_series_cr)
a = 0.05
#двусторонний критерий
print('Квантиль N(0,1) уровня', 1 - a/2, ':', norm.ppf(1-a/2))
print('|T| > z_(1-a/2) => отвергаем')

Значения статистик:  [-2.5505386250701143, 3.762208586308895, -2.5106679465399906, 1.0416609195243756]
Квантиль N(0,1) уровня 0.975 : 1.959963984540054
|T| > z_(1-a/2) => отвергаем


In [ ]:
#Вывод: строчки 3 и 4 - не Бернулли

In [ ]:
#2 определим какая из строк 1 и 2 соответвует Bern(.5)
#Будем действовать по критерию знаков, число единичек - m уже посчитано
from scipy.stats import binom
from math import fabs
import numpy as np
#Отвергаем при |m[i] - n/2| > ppf(a/2) (точный критерий)
a = 0.05
print('Первая строка: ', fabs(m[2]-n/2)/np.sqrt(15))
print('Вторая строка: ', fabs(m[3]-n/2)/np.sqrt(15))
print('Квантиль Binom(', n, '.5) уровня ', 1-a/2, ':', binom.ppf(1-a/2,n,.5, loc=-n/2))

Первая строка:  1.0327955589886444
Вторая строка:  0.0
Квантиль Binom( 60 .5) уровня  0.975 : 8.0


In [ ]:
#Отвергаем при |m[i] - n/2|/sqrt(n/4) > z_(1-a/2) (асимптотический критерий)
a = 0.05
print('Первая строка: ', fabs(m[0]-n/2)/sqrt(n/4))
print('Вторая строка: ', fabs(m[3]-n/2)/sqrt(n/4))
print('Квантиль N(0,1) уровня ', 1-a/2, ':', norm.ppf(1-a/2) + 1/sqrt(n))

Первая строка:  3.0983866769659336
Вторая строка:  0.0
Квантиль N(0,1) уровня  0.975 : 2.0890634294136348


In [ ]:
#Вывод: не можем отвергнуть гипотезу, что строкам соответствует Bern(.5)

In [ ]:
#3 Отделим ЦМ от нелучайной последовательности
#Найдем последовательность перемен знака:
delta = [[], [], [], []]
for i in range(4):
    for j in range(n-1):
        if X[i][j]!=X[i][j+1]:
            delta[i].append(1)
        else:
            delta[i].append(0)

ones = []
zeros = []
for i in range(4):
    s = 0
    for j in range(n-1):
        if delta[i][j] == 1:
            s += 1
    ones.append(s)
    zeros.append(n-1-s)

#теперь к delta применим Критерий серий

T_series_cr_1 = []
for i in range(4):
    s = 0
    for j in range(n-2):
        if delta[i][j] != delta[i][j+1]:
            s += 1
    T_series_cr_1.append((s + 1 - 2*zeros[i]*ones[i]/(n-1) - 1)/sqrt(2*zeros[i]*ones[i]*(2*zeros[i]*ones[i] - n+1)/((n-1)*(n-1)*(n-2))))

print('Значения статистик: ', T_series_cr_1)
a = 0.05
#двусторонний критерий
print('Квантиль N(0,1) уровня', 1 - a/2, ':', norm.ppf(1-a/2))
print('|T| > z_(1-a/2) => отвергаем гипотезу, что у нас ЦМ')


Значения статистик:  [0.2560035731172503, 0.5666837001147788, 2.806987350151077, 0.05015658172059708]
Квантиль N(0,1) уровня 0.975 : 1.959963984540054
|T| > z_(1-a/2) => отвергаем гипотезу, что у нас ЦМ


In [ ]:
#Вывод: четвертая строка - ЦМ, третья строка не ЦМ, значит она не случайная

In [ ]:
# хи квадратом найти Bern(.5)
# см записи к семинару

In [ ]:
#оценка параметров
Y = [X[0], X[1], delta[1]]
a = .05
z = norm.ppf(1-a/2)
estimate = []
residual = []
for i in range(3):
    s = 0
    len = 0
    for j in Y[i]:
        s += j
        len += 1
    estimate.append(s/len)
    residual.append(z*sqrt(s/len * (1-s/len)/len))

print('Первая строка: Bern(p), p примерно ', estimate[0], '+-', residual[0])
print('Вторая строка: Bern(p), p примерно ', estimate[1], '+-', residual[1])
print('Четвертая строка: ЦМ с вероятностью перехода п примерно ', estimate[2], '+-', residual[2])
print('Все ас.д.и. уровня',1-a)

Первая строка: Bern(p), p примерно  0.7 +- 0.11595303304540783
Вторая строка: Bern(p), p примерно  0.45 +- 0.12588096613612113
Четвертая строка: ЦМ с вероятностью перехода п примерно  0.7457627118644068 +- 0.11110715860661405
Все ас.д.и. уровня 0.95


In [ ]:
str = ['01 01 01 01 01 00 11 10 11 10 11 01 10 11 00 01 10 10 11 00 01 00 00 11 10 00 01 01 11 00']
v=[7, 10, 6, 7]
s=30
P=30/60
p=[(P**0)*((1-P)**(2-0)), (P**1)*((1-P)**(2-1)), (P**1)*((1-P)**(2-1)), (P**2)*((1-P)**(2-2))]
T=0
for i in range(4):
  T+=(v[i]-s*p[i])**2/(s*p[i])
print(T)
print(p)

import scipy.stats as stats
print(stats.chi2.ppf(0.95, df=2))


1.2000000000000002
[0.25, 0.25, 0.25, 0.25]
5.991464547107979
